In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.7 MB/s eta 0:00:00


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=5503633b687de58886b885e4d644c0895c6ad9291df93da6122dfb48892cfc8d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import requests
import xml.etree.ElementTree as ET
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

ModuleNotFoundError: ignored

In [ ]:
# Path to the XML file
xml_file_path = "/content/drive/MyDrive/Elsevier /LG LLM/topics2015A.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

In [ ]:
# Extract the clinical notes and reference summaries
clinical_notes = []
reference_summaries = []
for topic in root.findall("topic"):
    description = topic.find("description").text
    clinical_notes.append(description)
    summary = topic.find("summary").text
    reference_summaries.append(summary)

In [ ]:
print(len(clinical_notes))
len(reference_summaries)

In [ ]:
clinical_notes[0]

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

clinical_note_lengths = []
summary_lengths = []

for note, summary in zip(clinical_notes, reference_summaries):
    clinical_note_tokens = tokenizer.encode(note, add_special_tokens=False)
    summary_tokens = tokenizer.encode(summary, add_special_tokens=False)

    clinical_note_lengths.append(len(clinical_note_tokens))
    summary_lengths.append(len(summary_tokens))

average_clinical_note_tokens = sum(clinical_note_lengths) / len(clinical_note_lengths)
average_summary_tokens = sum(summary_lengths) / len(summary_lengths)

print("Average number of tokens in clinical notes:", average_clinical_note_tokens)
print("Average number of tokens in summaries:", average_summary_tokens)


Average number of tokens in clinical notes: 111.63333333333334
Average number of tokens in summaries: 32.93333333333333


In [ ]:
# Initialize BART tokenizer and model
#tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
#model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize clinical notes and evaluate
for i, note in enumerate(clinical_notes):
    inputs = tokenizer.encode(note, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, num_beams=4, max_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Clinical Note:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")




/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Clinical Note:
A 44 yo male is brought to the emergency room after multiple bouts of vomiting that has a "coffee ground" appearance. His heart rate is 135 bpm and blood pressure is 70/40 mmHg. Physical exam findings include decreased mental status and cool extremities. He receives a rapid infusion of crystalloid solution followed by packed red blood cell transfusion and is admitted to the ICU for further care.

Reference Summary:
A 44-year-old man with coffee-ground emesis, tachycardia, hypoxia, hypotension and cool, clammy extremities.

Generated Summary:
A 44 yo male is brought to the emergency room after multiple bouts of vomiting that has a "coffee ground" appearance. His heart rate is 135 bpm and blood pressure is 70/40 mmHg. Physical exam findings include decreased mental status and cool extremities. He receives a rapid infusion of crystalloid solution followed by packed red blood cell transfusion and is admitted to the ICU for further care.

BLEU Score: 8.380500118622762e-232
RO

In [ ]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.07298090103503917
Average ROUGE-1 Score: 0.3330499863920325
Average ROUGE-2 Score: 0.18633796987535006
Average ROUGE-L Score: 0.30102504791928664
Average METEOR Score: 0.46703674422170216


In [ ]:
bart large
Average BLEU Score: 0.1591846009150742
Average ROUGE-1 Score: 0.3877955304465891
Average ROUGE-2 Score: 0.27000199960889637
Average ROUGE-L Score: 0.35379814397744974

Bart Base
Average BLEU Score: 0.1783641498431823
Average ROUGE-1 Score: 0.405226409066793
Average ROUGE-2 Score: 0.2918551636750237
Average ROUGE-L Score: 0.36829885204737667

In [ ]:
#entire code

import requests
import xml.etree.ElementTree as ET
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

# Path to the XML file
xml_file_path = "/content/drive/MyDrive/Elsevier /LG LLM/topics2016.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extract the clinical notes and reference summaries
clinical_notes = []
reference_summaries = []
for topic in root.findall("topic"):
    description = topic.find("description").text
    clinical_notes.append(description)
    summary = topic.find("summary").text
    reference_summaries.append(summary)

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Initialize BLEU and ROUGE scorers

bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize clinical notes and evaluate
for i, note in enumerate(clinical_notes):
    inputs = tokenizer.encode(note, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, num_beams=4, max_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Clinical Note:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)